<a href="https://colab.research.google.com/github/Amith-kuppili/summarize.AI/blob/main/summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers accelerate datasets

In [ ]:
from datasets import load_dataset
# Corrected dataset name to a valid one on the Hugging Face Hub
multitimes = load_dataset("multi_news", split='test')

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
multitimes.to_pandas()

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a
...,...,...,...
287108,"The nine-year-old daughter of a black, unarmed...","Rumain Brisbon, 34, was killed after Phoenix p...",279a12d3ee37b8109cc192a9e88115a5a631fb06
287109,Legalising assisted suicide is a slippery slop...,"Theo Boer, a European assisted suicide watchdo...",b5bc9d404a9a5d890c9fc26550b67e6d8d83241f
287110,A group calling itself 'The Women of the 99 Pe...,Ohio congressman criticised for 'condoning the...,500862586f925e406f8b662934e1a71bbee32463
287111,Most men enjoy a good pint of lager or real al...,The Black Country Ale Tairsters have been to 1...,32a1f9e5c37a938c0c0bca1a1559247b9c4334b2


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
multi_news=multitimes.train_test_split(test_size=0.2)

In [ ]:
prefix='summarize'
def process_function(example):
  # Corrected to use 'article' column for input
  input=[prefix + ex for ex in example['article']]
  model_input=tokenizer(input,max_length=1024, truncation=True)
  # Corrected to use 'highlights' column for label
  label=tokenizer(text=example['highlights'],max_length=128, truncation=True)
  model_input["label"]=label['input_ids']
  return model_input

In [ ]:
tokenizer_multi_news=multi_news.map(process_function,batched=True)

Map:   0%|          | 0/229690 [00:00<?, ? examples/s]

Map:   0%|          | 0/57423 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq,AutoModelForSeq2SeqLM,Seq2SeqTrainingArguments,Seq2SeqTrainer
data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,model='t5-small')
model=AutoModelForSeq2SeqLM.from_pretrained('t5-small')

In [ ]:
training_args=Seq2SeqTrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    fp16=True,
    push_to_hub=False
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenizer_multi_news['train'],
    eval_dataset=tokenizer_multi_news['test'],
    data_collator=data_collator,

)



In [1]:
trainer.train()

NameError: name 'trainer' is not defined